An ensemble method = combining multiple models to get higher accuracy, stability, and better generalization than a single model.

Types of Ensemble Methods

Bagging (Bootstrap Aggregating)

Many models trained in parallel on random subsets.

Example: Random Forest

Boosting

Models trained sequentially, each fixing the previous one’s mistakes.

Examples: AdaBoost, Gradient Boosting, XGBoost

Stacking

Different models (like decision trees, SVM, logistic regression) are combined, and their outputs are fed into a meta-model for final prediction.

A decision stump is just a very simple decision tree with only one split (one level).

AdaBoost

Error Handling: Reweights misclassified samples.
Loss Function: No explicit loss; reduces classification error.
✅ Best for small datasets, interpretability, and imbalanced data.
❌ Sensitive to noise, weaker on complex data.

Gradient Boosting (GBM)

Error Handling: Fits to residuals using gradient descent.
Loss Function: Explicit, differentiable loss functions.
✅ Best for general regression/classification tasks.
❌ Slower training compared to AdaBoost/XGBoost.

XGBoost

Error Handling: Same as GBM but optimized.
Regularization: Built-in (L1, L2) → prevents overfitting.
Optimization: Parallelization + scalability.
✅ Best for large datasets, high performance, competitions.
❌ More complex to tune.

XGBoost achieves its speed and efficiency through a combination of algorithmic and engineering optimizations:
Parallel Processing:
Handling Missing Data:
Tree Pruning (Depth-wise growth with post-pruning):
Column Block (Cache-aware) and Parallel Learning:
Regularization in XGBoost:
Out-of-Core Computing:
(For datasets larger than available memory, XGBoost can use disk-based data structures, enabling it to handle massive datasets efficiently).

Why Cross-Validation is Important in Boosting?

Prevents Overfitting
Helps in Hyperparameter Tuning

Parameters like:
Number of estimators (trees)
Learning rate
Tree depth (in case of stumps, max_depth=1)
Using k-fold CV lets us select the best combination that balances bias vs variance.

Avoids Bias from One Train-Test Split
A single train-test split might give lucky/unlucky results.
CV averages across folds → more reliable performance estimate.
Model Selection

Key Hyperparameters in AdaBoost, Gradient Boosting, XGBoost; Interaction of Learning Rate & Number of Estimators
AdaBoost

n_estimators: number of weak learners.
earning_rate: shrinkage per learner.

Base learner parameters (e.g., tree depth).
Gradient Boosting (GBM)
n_estimators
learning_rate
max_depth
min_samples_split, min_samples_leaf
Subsampling parameters—e.g., row and column sampling.

XGBoost
n_estimators
learning_rate (eta)
max_depth
alpha (L1 regularization), lambda (L2)
min_child_weight, gamma, subsample, colsample_bytree


Over/Underfitting & Curves
Learning/validation curves help visualize model behavior and guide tuning.